In [13]:
import json
import os
from google.cloud import storage
import cromwell_manager as cwm

with open(os.path.expanduser('~/.ssh/mint_cromwell_config.json')) as f:
    cromwell_server = cwm.Cromwell(**json.load(f))

storage_client = storage.Client(project='broad-dsde-mint-dev')

os.environ['wdltool'] = '/Users/ajc/google_drive/software/wdltool-0.14.jar'

In [15]:
cat example_test_inputs.json

{
  "Optimus.fastq_inputs": [
    [
      "gs://broad-dsde-mint-dev-teststorage/10x/demo/fastqs/pbmc8k_S1_L007_R1_001.fastq.gz",
      "gs://broad-dsde-mint-dev-teststorage/10x/demo/fastqs/pbmc8k_S1_L007_R2_001.fastq.gz",
      "gs://broad-dsde-mint-dev-teststorage/10x/demo/fastqs/pbmc8k_S1_L007_I1_001.fastq.gz"
    ],
    [
      "gs://broad-dsde-mint-dev-teststorage/10x/demo/fastqs/pbmc8k_S1_L007_R1_001.fastq.gz",
      "gs://broad-dsde-mint-dev-teststorage/10x/demo/fastqs/pbmc8k_S1_L007_R2_001.fastq.gz",
      "gs://broad-dsde-mint-dev-teststorage/10x/demo/fastqs/pbmc8k_S1_L007_I1_001.fastq.gz"
    ]
  ],
  "Optimus.whitelist": "gs://broad-dsde-mint-dev-teststorage/10x/whitelist/737K-august-2016.txt",
  "Optimus.tar_star_reference": "gs://broad-dsde-mint-dev-teststorage/demo/star.tar",
  "Optimus.sample_id": "pbmc8k_test",
  "Optimus.annotations_gtf": "gs://broad-dsde-mint-dev-teststorage/reference/hg19_ds/GSM1629193_hg19_ERCC.gtf.gz",
  "Optimus.ref_genome_fasta": "gs://broad-dsde-

In [19]:
inputs_json = {
    "Optimus.fastq_inputs": [
      [
          "gs://broad-dsde-mint-dev-teststorage/10x/pbmc8k/fastqs/pbmc8k_S1_L007_R1_001.fastq.gz",
          "gs://broad-dsde-mint-dev-teststorage/10x/pbmc8k/fastqs/pbmc8k_S1_L007_R2_001.fastq.gz",
          "gs://broad-dsde-mint-dev-teststorage/10x/pbmc8k/fastqs/pbmc8k_S1_L007_I1_001.fastq.gz"
      ],
      [
          "gs://broad-dsde-mint-dev-teststorage/10x/pbmc8k/fastqs/pbmc8k_S1_L008_R1_001.fastq.gz",
          "gs://broad-dsde-mint-dev-teststorage/10x/pbmc8k/fastqs/pbmc8k_S1_L008_R2_001.fastq.gz",
          "gs://broad-dsde-mint-dev-teststorage/10x/pbmc8k/fastqs/pbmc8k_S1_L008_I1_001.fastq.gz"
      ]
    ],
    "Optimus.whitelist": "gs://broad-dsde-mint-dev-teststorage/10x/whitelist/737K-august-2016.txt",
    "Optimus.tar_star_reference": "gs://broad-dsde-mint-dev-teststorage/demo/star.tar",
    "Optimus.sample_id": "pbmc8k_test",
    "Optimus.annotations_gtf": "gs://broad-dsde-mint-dev-teststorage/reference/hg19_ds/GSM1629193_hg19_ERCC.gtf.gz",
    "Optimus.ref_genome_fasta": "gs://broad-dsde-mint-dev-teststorage/demo/chr21.fa"
}

wdl = "Optimus.wdl"

workflow_dependencies = {
    'Optimus.wdl': '../optimus/Optimus.wdl',
    "StarAlignBamSingleEnd.wdl": "../pipelines/tasks/StarAlignBamSingleEnd.wdl",
    "FastqToUBam.wdl": "../pipelines/tasks/FastqToUBam.wdl",
    "Attach10xBarcodes.wdl": "../pipelines/tasks/Attach10xBarcodes.wdl",
    "SplitBamByCellBarcode.wdl": "../pipelines/tasks/SplitBamByCellBarcode.wdl",
    "TagGeneExon.wdl": "../pipelines/tasks/TagGeneExon.wdl",
    "CorrectUmiMarkDuplicates.wdl": "../pipelines/tasks/CorrectUmiMarkDuplicates.wdl",
    "CollectMultiplePicardMetrics.wdl": "../pipelines/tasks/CollectMultiplePicardMetrics.wdl",
    "MergeSortBam.wdl": "../pipelines/tasks/MergeSortBam.wdl",
    "CreateCountMatrix.wdl": "../pipelines/tasks/CreateCountMatrix.wdl",
    "AlignTagCorrectUmis.wdl": "../optimus/AlignTagCorrectUmis.wdl"    
}

In [39]:
cwm.Workflow.validate(
    wdl=wdl,
    inputs_json=inputs_json,
    storage_client=storage_client,
    workflow_dependencies=workflow_dependencies,
    cromwell_server=cromwell_server)

CWM:2018-01-08 20:07:42.953779:creating temporary directory
CWM:2018-01-08 20:07:42.954540:writing dependencies
CWM:2018-01-08 20:07:42.968711:writing wdl
CWM:2018-01-08 20:07:42.969804:running wdltool validate
Error: Unable to access jarfile /Users/ajc/google_drive/software/wdltool-0.14.jar

CWM:2018-01-08 20:07:43.482615:checking docker image humancellatlas/picard:2.10.10... OK.
CWM:2018-01-08 20:07:43.777050:checking docker image humancellatlas/dropseqtools:1.12... OK.
CWM:2018-01-08 20:07:45.049217:checking docker image humancellatlas/samtools:1.3.1... OK.
CWM:2018-01-08 20:07:45.765581:checking docker image humancellatlas/python3-scientific:0.1.5... OK.
CWM:2018-01-08 20:07:45.996618:checking docker image us.gcr.io/broad-gotc-prod/genomes-in-the-cloud:2.3.3-1513176735... not found. Is image private?
CWM:2018-01-08 20:07:46.299912:checking docker image humancellatlas/star:2.5.3a-40ead6e... OK.


In [40]:
wf = cwm.Workflow.from_submission(
    wdl=wdl,
    inputs_json=inputs_json,
    storage_client=storage_client,
    workflow_dependencies=workflow_dependencies,
    cromwell_server=cromwell_server)

In [46]:
wf.status

{'id': 'fb314e86-5943-41cb-9ec9-361ed92f4833', 'status': 'Succeeded'}

In [44]:
cromwell_server.metadata(wf.id, open_browser=True)

<Response [200]>

In [4]:
wf.timing()